In [2]:
import cv2
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt

proj_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
model = YOLO(os.path.join(proj_path, 'scripts/train_model/train2/weights/best.pt'))
img = cv2.imread(os.path.join(proj_path, 'data/photos/Donald_Trump_mug_shot.jpg'))

In [3]:
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = model(img_rgb)


0: 640x640 1 face, 165.2ms
Speed: 5.0ms preprocess, 165.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


In [4]:
results_boxes = results[0].boxes
x1, y1, x2, y2 = list(map(int, results_boxes.xyxy[0].tolist()))

In [5]:
import dlib


predictor = dlib.shape_predictor(os.path.join(proj_path, 'data/weights/shape_predictor_68_face_landmarks.dat'))


In [6]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
face = dlib.rectangle(x1, y1, x2, y2)
landmarks = predictor(gray, face)
aligned_face = dlib.get_face_chip(img, landmarks)
face_recognizer = dlib.face_recognition_model_v1(os.path.join(proj_path, 'data/weights/dlib_face_recognition_resnet_model_v1.dat'))
face_feature = face_recognizer.compute_face_descriptor(aligned_face)


In [ ]:
import numpy as np
from annoy import AnnoyIndex

face_feature = np.array(face_feature)
dim = 128 
index = AnnoyIndex(dim, 'angular')
index.add_item(0, face_feature)


In [10]:
index.build(10)

True

In [11]:
neighbors = index.get_nns_by_vector(face_feature, 5)  # 查找 5 个最近邻
print(neighbors)

[0]
